# Example notebook: Australian census data

Example notebook to show how to download, process and display geopackaged Australian census data.

Australian census data downloaded from:
 - https://www.abs.gov.au/census/find-census-data/geopackages

Using the following settings:
 - 2021 census year
 - All state and territories
 - G01 tables (population counts)
 - 2020 GDA (geo data specifications)

## Imports Libraries & Set Globals

In [1]:
# Import libraries
import geopandas as gpd
import plotly.graph_objects as go

# Set globals
pass

# Set Variables & Load Data

In [2]:
# Set variables
census_year = '2021'
geo_type = 'AUST'
data_topic = 'G01'
gda_type = '2020'
map_type = 'SA4'

# Load data
file_base = 'C:/Users/zlatt/Documents/data/australian-census-data'
file_directory = f'Geopackage_{census_year}_{data_topic}_{geo_type}_GDA{gda_type}'
file_name = f'{data_topic}_{geo_type}_GDA{gda_type}.gpkg'

file_path = f'{file_base}/{file_directory}/{file_name}'
layer = f'{data_topic}_{map_type}_{census_year}_{geo_type}'

census_gdf = gpd.read_file(
    filename=file_path,
    layer=layer
)

# Process data
columns = [
    f'{map_type}_CODE_{census_year}',
    f'{map_type}_NAME_{census_year}',
    'Tot_P_P',
    'AREA_ALBERS_SQKM',
    'geometry'
]
census_gdf = census_gdf[columns]

rename_columns = {
    f'{map_type}_CODE_{census_year}': 'Code', 
    f'{map_type}_NAME_{census_year}': 'Name',
    'Tot_P_P': 'Population',
    'AREA_ALBERS_SQKM': 'Area',
    'geometry': 'geometry'
}
census_gdf = census_gdf.rename(columns=rename_columns)

census_gdf = census_gdf[
    census_gdf['geometry'] != None
]
census_gdf = census_gdf.reset_index(drop=True)

column_types = {
    'Code': 'str',
    'Name': 'str',
    'Population': 'int',
    'Area': 'float',
    'geometry': 'geometry'
}
census_gdf = census_gdf.astype(column_types)

## Create Figure and Display Data

In [3]:
# Create figure data
figure_df = census_gdf.copy()
figure_df = figure_df.set_index('Name')
figure_geojson = figure_df.__geo_interface__

# Create figure
figure = go.Figure()

# Create and add choroplethmap trace
trace = go.Choroplethmapbox(
    name='Census',
    geojson=figure_geojson,
    locations=figure_df.index,
    z=figure_df['Population'],
    marker_opacity=0.5
)
figure = figure.add_trace(trace)

# Create and add map layout
layout = go.Layout(
    mapbox=dict(
        style="carto-positron",
        center=dict(
            lat=-25,
            lon=130
        ),
        zoom=2,
        bounds=dict(
            west=85,
            east=185,
            north=0,
            south=-50
        )
    ),
    autosize=True,
    margin=dict(
        l=0,
        r=0,
        t=0,
        b=0
    ),
    height=650,
    width=1300
)
figure = figure.update_layout(layout)

In [4]:
# Show figure
#figure.show()
figure.write_html(f"{file_base}/figure.html")